# Experiment Configuration and Execution

In this tutorial we show how to configure and execute an experiment in OpenSBT to test an agent. We show the configuration of the experiment for testing a simplistic agent in a simplified simulator (no GPU required).

_Note: This example is meant for introducing OpenSBT. An application with a high fidelity simulation and a *real-world* SUT is provided [here](https://git.fortiss.org/opensbt/opensbt-core/-/blob/main/doc/jupyter/06_Example_CARLA.ipynb)._

## Toy Example

We refer to this example as Toy Example. The example contains an AEB agent and a pedestrian. Both actors move linear, while the pedestrian has a fixed trajectory and fixed velocity, the ego agent has a predefined path but can modify his velocity. That means, ego brakes when he detects other actors within a predefined distance. We want to test the AEB agent in violating the safety distance of 1m. For this we want to vary the initial velocities of both actors as well the orientation of the pedestrian crossing egos' lane.

In particlar, the pedestrian has a speed in the range of [0.2, 3] m/s, the ego [1, 10] m/s, while the orientation range is [0,180] degrees.

## Defining An Experiment

To define an experiment we need to instantiate [`ADASProblem`](https://git.fortiss.org/opensbt/opensbt-core/-/blob/main/problem/adas_problem.py). `ADASProblem` holds:

- the **scenario**, which can be e.g., an OpenSCENARIO file, or file of another format supported by simulator,
- the **search variables**, whose values are altered to produce (different) scenarios througout the search,
- the **search space**, represented by the lower and upper bounds as arrays,
- the **fitness function**, which evaluates how 'fit' a test case is,
- the **criticality function**, which represents the safety requirements/testing oracle,
- the **simulate function** to trigger the simulator via this function,
- the **simulator-related variables**, i.e., simulation time, sampling time, toggle for visualization.

_Note, that for now the SUT is triggered via the Simulater Interface, as we assume that the SUT is embedded into the simulator. We are working on an update where the SUT can be defined uncoupled from the simulator._

In [ ]:
cd opensbt-core

In [ ]:
import pymoo

from model_ga.individual import IndividualSimulated
pymoo.core.individual.Individual = IndividualSimulated

from model_ga.population import PopulationExtended
pymoo.core.population.Population = PopulationExtended

from model_ga.result  import SimulationResult
pymoo.core.result.Result = SimulationResult

from model_ga.problem import SimulationProblem
pymoo.core.problem.Problem = SimulationProblem

from algorithm.nsga2_optimizer import NsgaIIOptimizer
from experiment.search_configuration import DefaultSearchConfiguration

import os
from evaluation.fitness import *
from problem.adas_problem import ADASProblem
from problem.pymoo_test_problem import PymooTestProblem
from experiment.experiment_store import *
from algorithm.algorithm import *
from evaluation.critical import *
from simulation.dummy_simulation import DummySimulator


problem = ADASProblem(
                      problem_name="DummySimulatorProblem",
                      scenario_path="scenarios/dummy_scenario.xosc",
                      xl=[0, 1, 0, 0.2],
                      xu=[180, 10,180, 3],
                      simulation_variables=[
                          "orientation_ego",
                          "velocity_ego",
                          "orientation_ped",
                          "velocity_ped"],
                      fitness_function=FitnessMinDistanceVelocity(),
                      critical_function=CriticalAdasDistanceVelocity(),
                      simulate_function=DummySimulator.simulate,
                      simulation_time=10,
                      sampling_time=0.25
                      )

We have two options to execute the testing experiment: using the console or using the code directly. 

## Experiment Execution (Code)

To start search without console, we instantiate the search algorithms `Optimizer` and call its `run` method to start the search. When the search has finished, we write the results by calling an output method (line 9). The results will be written in the default folder name `results`, if no folder name is passed.

In [ ]:
import pymoo

from model_ga.individual import IndividualSimulated
pymoo.core.individual.Individual = IndividualSimulated

from model_ga.population import PopulationExtended
pymoo.core.population.Population = PopulationExtended

from model_ga.result  import SimulationResult
pymoo.core.result.Result = SimulationResult

from model_ga.problem import SimulationProblem
pymoo.core.problem.Problem = SimulationProblem

from algorithm.nsga2_optimizer import NsgaIIOptimizer

from utils import log_utils

log_utils.setup_logging("./log.txt")

# Set search configuration
config = DefaultSearchConfiguration()
config.n_generations = 50
config.population_size = 20

# Instantiate search algorithm
optimizer = NsgaIIOptimizer(
                            problem=problem,
                            config= config)

# Run search
res = optimizer.run()

# Write results
res.write_results(params = optimizer.parameters)

We can then for instance inspect the visualization of test cases that has been found during search. Note, that visualizations of the "fitness"-optimal scenarios are stored in the "gif" folder.

In [ ]:
import pandas as pd
from pathlib import Path
import os

exp_folder = os.getcwd() + f"/results/DummySimulatorProblem/NSGA2/"
paths = sorted(Path(exp_folder).iterdir(), key=os.path.getmtime)
results_path = str(paths[-1])

from IPython.display import Image
import os

n = 4 # show max n images
folder_gifs = results_path + os.sep + "gif"
i = 0
for f in os.listdir(folder_gifs):
    fpath = folder_gifs + os.sep + f
    if os.path.isfile(fpath) and i < n:
      img = Image(open(fpath,'rb').read(), width=300, height=300)
      display(img)
      i = i+1

## Experiment Execution (Console)

To start the search through the console we need to create an `Experiment` instance. The experiment instance receives the following inputs:

- the experiment name (type `str`)
- the created problem (type `ADASProblem`) 
- the search algorithm (type `AlgorithmType`)
- the search configuration (type `SearchConfiguration`)

Note, that the algorithm that is passed is an enumeration value which need to be registed beforehand via the `algorithm.py` script.

```python
from experiment.experiment import Experiment
from algorithm.algorithm import AlgorithmType
from experiment.search_configuration import DefaultSearchConfiguration

experiment = Experiment(name="5",
                        problem=problem,
                        algorithm=AlgorithmType.NSGAII,
                        search_configuration=DefaultSearchConfiguration())
```

Then we register the experiment so that we can use it later via console.

```python
from experiment.experiment_store import experiments_store
experiments_store.register(experiment)
```

To start search via console we run the experiment with the name "5":

We can change the experiment parameters, such e.g., lower and upper bounds of the search parameters, the search time, iteration size or population size by using flags. In the following execution, we restrict the search by selecting a maximal angle of 90 degrees and use only 10 iterations for the genetic operations.

In [ ]:
!python -m opensbt.run -e 5 -min 0 1 90 0.2 -max 90 10 90 3 \
-m  "orientation_ped" "velocity_ped" "orientation_ped" "velocity_ped" -i 10

We can then inspect some of the optimal test inputs that have been found:

In [ ]:
import pandas as pd
from pathlib import Path
import os

exp_folder = os.getcwd() + f"/results/DummySimulatorProblem/NSGA2/"
paths = sorted(Path(exp_folder).iterdir(), key=os.path.getmtime)
results_path = str(paths[-1])

from IPython.display import Image
import os

n = 4 # show max n images
folder_gifs = results_path + os.sep + "gif"
i = 0
for f in os.listdir(folder_gifs):
    fpath = folder_gifs + os.sep + f
    if os.path.isfile(fpath) and i < n:
      img = Image(open(fpath,'rb').read(), width=300, height=300)
      display(img)
      i = i+1

## Flags

The complete list of flags is provided below.

```
 -h, --help            show this help message and exit
  -e EXP_NUMBER         Name of existing experiment to be used. (show all experiments via -info)].
  -i N_GENERATIONS      Number of generations to perform.
  -n SIZE_POPULATION    The size of the initial population of scenario candidates.
  -a ALGORITHM          The algorithm to use for search (Number registered via AlgorithmType)
  -t MAXIMAL_EXECUTION_TIME
                        The time to use for search.
  -f SCENARIO_PATH      The path to the scenario description file.
  -min VAR_MIN [VAR_MIN ...]
                        The lower bound of each search parameter.
  -max VAR_MAX [VAR_MAX ...]
                        The upper bound of each search parameter.
  -m DESIGN_NAMES [DESIGN_NAMES ...]
                        The names of the variables to modify.
  -o RESULTS_FOLDER     The name of the folder where the results of the search are stored (default: /results/single/)
  -v                    Whether to use the simuator's visualization. This feature is useful for debugging and demonstrations, however it reduces the search performance.
  -info                 Names of all defined experiments.
```
